In [1]:

import os

In [2]:
os.environ['EPICS_CA_ADDR_LIST'] += ' 10.0.38.35'
os.environ['EPICS_CA_ADDR_LIST']

' 10.128.1.12:5064 10.128.1.12:5070 10.128.1.13:5064 10.128.1.13:5068 10.128.1.13:5070 10.128.1.54 10.128.1.55  10.0.38.29  10.0.38.37 10.0.38.39 10.0.38.52 10.0.38.35 10.0.38.143  10.0.38.31 10.0.38.150  10.31.74.16 10.0.38.83 10.0.38.72 10.0.38.149 10.0.38.36  10.30.14.19 10.30.13.22 10.39.50.46 10.31.54.38 10.32.74.44  10.10.10.124  10.0.28.64   10.0.38.35'

In [3]:
import numpy as np
from datetime import datetime
import time
from siriuspy.search import IDSearch
from apsuite.commisslib.meas_bpms_signals import AcqBPMsSignals
from siriuspy.devices import HLFOFB, SOFB, VPU, IDFF
import epics

In [4]:
def initialize_vpu(beamline):
    # Search ID
    devnamevpu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    vpu = VPU(devname=devnamevpu)

    # Disable beamline control
    vpu.cmd_beamline_ctrl_disable()
    print('beamline control: ', vpu.is_beamline_ctrl_enabled)

    # Set gap speed
    vpu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(vpu.gap_speed))

    # Set gap to parked condition
    vpu.set_gap(vpu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(vpu.gap))

    return vpu

def move_vpu_gap(vpu:VPU, gap, timeout, verbose=False):
    vpu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(vpu.gap)) if verbose else 0
    if vpu.cmd_move_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while vpu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(vpu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(vpu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False
    
def configure_acquisition_params(orbacq, timeout=40, nrptsbefore=10000, nrptsafter=100000):
    """."""
    params = orbacq.params
    params.signals2acq = 'XY'
    params.acq_rate = 'FAcq' 
    params.timeout = 40

    params.nrpoints_before = nrptsbefore  # Default: 1000
    params.nrpoints_after = nrptsafter  # Default: 10000
    params.acq_repeat = False
    params.trigbpm_delay = None
    params.trigbpm_nrpulses = 1

    params.timing_event = 'Study' 
    params.event_mode = 'External'
    params.event_delay = None
    params.do_pulse_evg = False  # Default: False
    print('--- orbit acquisition configuration ---')
    print(params)


def send_ext_trigger(evt):
    evt.cmd_external_trigger()
    return time.time()


def acquire_data_moving(orbacq, vpu, start_gap, end_gap):
    """."""
    fambpms = orbacq.devices["fambpms"]
    ret = orbacq.prepare_bpms_acquisition()
    tag = orbacq._bpm_tag(idx=abs(int(ret))-1)
    if ret < 0:
        print(tag + " did not finish last acquisition.")
    elif ret > 0:
        print(tag + " is not ready for acquisition.")

    fambpms.reset_mturn_initial_state()
    orbacq.trigger_timing_signal()
    evt = orbacq._get_event(orbacq.params.timing_event)

    move_vpu_gap(vpu, gap=start_gap, timeout=3, verbose=True)
    vpu.set_gap(end_gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(vpu.gap))

    pv_gap = vpu.pv_object(vpu.PARAM_PVS.KPARAM_MON)
    pv_gap.auto_monitor = True
    id_gaps, id_tss = [], []
    def pv_gap_callback(pvname, value, **kwargs):
        id_gaps.append(value)
        id_tss.append(time.time())
    idx_cb = pv_gap.add_callback(pv_gap_callback)
    time.sleep(1)

    t0_ = send_ext_trigger(evt)
    time.sleep(0.3)
    vpu.cmd_move_start(timeout=3)

    time0 = time.time()
    ret = fambpms.wait_update_mturn(timeout=orbacq.params.timeout)
    print(f"it took {time.time()-time0:02f}s to update bpms")
    if ret != 0:
        print("There was a problem with acquisition")
        if ret > 0:
            tag = orbacq._bpm_tag(idx=int(ret)-1)
            pos = fambpms.mturn_signals2acq[int((ret % 1) * 10) - 1]
            print("This BPM did not update: " + tag + ", signal " + pos)
        elif ret == -1:
            print("Initial timestamps were not defined")
        elif ret == -2:
            print("Signals size changed.")
        return
    orbacq.data = orbacq.get_data()
    pv_gap.remove_callback(idx_cb)
    pv_gap.auto_monitor = False
    orbacq.data['vpu_data'] = dict(
        id_gaps=id_gaps, id_timestamps=id_tss, trig_timestamp=t0_
    )
    

def measure(orbacq, vpu, start_gap, end_gap):
    """."""
    init_state = orbacq.get_timing_state()
    orbacq.prepare_timing()
    print((
        'Please check BPM acquisition status, or wait a few seconds, '
        ' before triggering event!'))
    try:
        acquire_data_moving(orbacq, vpu, start_gap, end_gap)
    except Exception as e:
        print(f"An error occurred during acquisition: {e}")
    # Restore initial timing state, regardless acquisition status
    orbacq.recover_timing_state(init_state)
    return orbacq.data is not None


def initialize(timeout, nrptsbefore, nrptsafter):
    """."""
    orbacq = AcqBPMsSignals(isonline=True)
    configure_acquisition_params(orbacq, timeout, nrptsbefore, nrptsafter)
    print('--- orbit acquisition connection ---')
    if not orbacq.wait_for_connection(timeout=100):
        raise RuntimeError('Orbacq did not connect.')
    return orbacq


def create_devices(beamline):
    """."""
    sofb = SOFB(SOFB.DEVICES.SI)
    fofb = HLFOFB(HLFOFB.DEVICES.SI)
    
    # Search ID
    devname = IDSearch.conv_beamline_2_idname(beamline=beamline)
    vpu = VPU(devname=devname)
    devices = sofb, fofb
    for dev in devices:
        if not dev.wait_for_connection(timeout=5):
            raise RuntimeError(f'{dev.devname:s} did not conect!')

    return sofb, fofb, vpu


def read_feedback_status(devs, orbacq):
    """."""
    sofb, fofb, _ = devs
    orbacq.data['sofb_loop_state'] = sofb.autocorrsts
    orbacq.data['fofb_loop_state'] = fofb.loop_state


In [5]:
timeout = 40
nrptsbefore=10000
nrptsafter=90000

beamline='CARNAUBA'
devs = create_devices(beamline)
orbacq = initialize(timeout, nrptsbefore, nrptsafter)

--- orbit acquisition configuration ---
trigbpm_delay              = same       (current value will not be changed)
trigbpm_nrpulses           =         1  
do_pulse_evg               = False      
timing_event               = Study      
event_delay                = same       (current value will not be changed)
event_mode                 = External   
timeout                    = 40.000000  [s]
nrpoints_before            =     10000  
nrpoints_after             =     90000  
acq_rate                   = FAcq       
acq_repeat                 =         0  
signals2acq                = XY         

--- orbit acquisition connection ---


In [8]:
start_gap = 9.7
devs[-1].set_gap_speed(1)
devs[-1].set_gap(start_gap)
devs[-1].cmd_move_start()

end_gap = 25
gap_speed = 1.0

vpu = devs[-1]
fofb = devs[1]
loopstate = fofb.auto_monitor_status['LoopState-Sel']
loopstate = 'ON' if loopstate is True else 'OFF'

ffstate = epics.caget('SI-06SB:BS-IDFF-CC:LoopState-Sts')
ffstate = 'ON' if ffstate == 1 else 'OFF'

In [9]:
vpu.set_gap_speed(gap_speed)
time.sleep(0.5)

filename_prefix = 'VPU29_' + beamline + '_{:.1f}-{:.1f}_mm'.format(start_gap, end_gap)
filename_prefix += 'FF{}_FB{}_gspeed_{:.3f}'.format(ffstate, loopstate, gap_speed)

if measure(orbacq, vpu, start_gap, end_gap):
    read_feedback_status(devs, orbacq)
    now = datetime.now()
    str_rate = f'FAcq_rate_'
    str_now = now.strftime('%Y-%m-%d_%Hh%Mm%Ss')
    filename = filename_prefix + str_rate + str_now
    orbacq.save_data(filename, overwrite=False)
    print(f'\nData saved at {str_now:s}')
else:
    print('\nData NOT saved!')

Please check BPM acquisition status, or wait a few seconds,  before triggering event!
Gap-RB 9.700 mm
Undulator is moving...
Gap 9.700 mm reached.
Gap-RB 25.000 mm
it took 26.030848s to update bpms

Data saved at 2025-06-16_20h39m57s


In [21]:
import glob
filenames = glob.glob('*.pickle')
print(filenames)

['IVU18_EMA_14.0-4.2_mm_gspeed_0.500FAcq_rate_2024-11-01_10h10m34s.pickle', 'IVU18_EMA_4.2-24.0_mm_gspeed_1.000FAcq_rate_2024-11-01_09h57m27s.pickle', 'IVU18_EMA_4.2-24.0_mm_gspeed_1.000FAcq_rate_2024-10-31_17h59m46s.pickle', 'IVU18_EMA_4.2-24.0_mm_gspeed_1.000FAcq_rate_2024-11-01_10h02m00s.pickle', 'orbit_distortions_corrs_IVU_EMA_iter0.pickle', 'IVU18_EMA_24.0-17.0_mm_gspeed_0.500FAcq_rate_2024-10-31_08h58m51s.pickle', 'ref_orb.pickle', 'IVU18_EMA_24.0-4.2_mm_gspeed_1.000FAcq_rate_2024-11-01_09h54m42s.pickle', 'orbit_distortions_IVU_EMA_iter0.pickle', 'orbit_distortions_IVU_PAINEIRA_iter0.pickle', 'orbit_distortions_corrs_IVU_PAINEIRA_iter0.pickle', 'IVU18_EMA_10.0-4.2_mm_gspeed_1.000FAcq_rate_2024-10-31_16h35m20s.pickle', 'IVU18_EMA_10.0-4.2_mm_gspeed_1.000FAcq_rate_2024-10-31_11h01m00s.pickle', 'IVU18_PAINEIRA_24.0-4.2_mm_gspeed_1.000FAcq_rate_2024-10-31_18h12m07s.pickle', 'IVU18_EMA_24.0-4.2_mm_gspeed_0.500FAcq_rate_2024-11-01_10h09m09s.pickle', 'IVU18_EMA_4.2-10.0_mm_gspeed_1.000